In [32]:
from pgmpy.models import BayesianModel
from pgmpy.estimators import HillClimbSearch, K2Score, ExhaustiveSearch
from pgmpy.sampling import BayesianModelSampling
import pandas as pd
import numpy as np
from pgmpy.inference.CausalInference import CausalInference

In [33]:
# 連続値データの読み込み
data = pd.read_excel('../data/230616_AI用qPCRデータ.xlsx', index_col=0)
data.head()

,glk,agp,yihX,pgm,ptsG,malx,crr,galM,yihR,yeaD,...,tpiA,gapA,pgk,gpmA,ytjc,gpmM,eno,pykF,pykA,ppsA
gene,,,,,,,,,,,,,,,,,,,,,
1,0.000004,2.982088e-05,0.000004,1.512045e-05,0.000015,9.403729e-07,0.000122,0.000015,4.792578e-07,0.000007,...,0.000008,0.000121,1.193258e-07,0.000119,0.000004,0.000008,0.000015,0.000030,0.000015,0.000030
2,0.000008,3.062644e-05,0.000016,2.940716e-08,0.000015,3.847846e-06,0.000124,0.000015,3.848767e-06,0.000015,...,0.000015,0.000123,2.377373e-07,0.000123,0.000008,0.000016,0.000031,0.000061,0.000031,0.000061
3,0.000008,7.608227e-06,0.000016,1.396326e-08,0.000031,9.615077e-07,0.000245,0.000015,4.864569e-07,0.000031,...,0.000015,0.000122,9.666787e-07,0.000122,0.000008,0.000016,0.000062,0.000062,0.000031,0.000062
4,0.000004,1.513944e-05,0.000015,1.225795e-07,0.000015,4.807837e-07,0.000124,0.000015,1.207673e-07,0.000008,...,0.000015,0.000061,2.410427e-07,0.000061,0.000004,0.000008,0.000031,0.000062,0.000031,0.000062
5,0.000008,4.768491e-07,0.000031,6.105349e-08,0.000031,9.593193e-07,0.000124,0.000015,2.428970e-07,0.000016,...,0.000016,0.000124,4.795912e-07,0.000124,0.000008,0.000008,0.000031,0.000062,0.000031,0.000062


In [34]:
# ベイジアンネットワークの構造探索
# hc = HillClimbSearch(data)
hc = ExhaustiveSearch(data, scoring_method=K2Score(data))
# best_network = hc.estimate(scoring_method=K2Score(data))
best_network = hc.estimate()
edges = best_network.edges()
nodes = best_network.nodes()
print(edges)
print(nodes)

KeyboardInterrupt: 

In [29]:
# from causalnex.structure import StructureModel
# from causalnex.plots import plot_structure

# sm_cancer_true = StructureModel()

# sm_cancer_true.add_edges_from(edges)
# viz = plot_structure(
#     sm_cancer_true,)

# viz.show('./n.html')

In [5]:
# DAGオブジェクトからBayesianModelオブジェクトへ変換
best_model_bayesian = BayesianModel(best_network)

/opt/anaconda3/envs/py3108/lib/python3.10/site-packages/pgmpy/models/BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


In [6]:
# 探索したネットワークのエッジからネットワークのモデルを構築してCPDを求める
model = BayesianModel(list(edges))
# 独立なノードが入っていない場合にも対応
model.add_nodes_from(list(nodes))
model.fit(data)  # cpds を計算
cpds = model.get_cpds()
# for cpd in cpds:
#     print(cpd, '\n')

In [7]:
for cpd in cpds:
    best_model_bayesian.add_cpds(cpd)

In [8]:
# パラメータ推定（確率分布の推定）
sampler = BayesianModelSampling(best_model_bayesian)
samples = sampler.forward_sample(size=1000)

  0%|          | 0/33 [00:00<?, ?it/s]

In [15]:
# from scipy.stats import norm
# import matplotlib.pyplot as plt
# import seaborn as sns
# sns.set()

# # 可視化関数
# def norm_plot(node, min_value, max_value, mu, std):
#     X = np.arange(start=min_value, stop=max_value, step=0.000000001)

#     # pdfで確率密度関数を生成
#     norm_pdf = norm.pdf(x=X, loc=mu, scale=std)

#     fig = plt.figure(figsize=(15, 8))

#     plt.plot(X, norm_pdf)
#     plt.title(node)
#     plt.show()
#     fig.savefig('./output/pdf/{}.png'.format(node))

In [16]:
# # 推定されたパラメータの確率分布を表示
# for node, min_value, max_value in zip(best_model_bayesian.nodes(), data.min(), data.max()):
#     variable_samples = samples[node]
#     variable_mean = np.mean(variable_samples)
#     variable_std = np.std(variable_samples)
#     print(f"{node}: Normal(mu={variable_mean}, sd={variable_std})")
#     norm_plot(node, min_value, max_value, variable_mean, variable_std)

In [ ]:
from pyvis import network as net

edge_list = list(best_network.edges())
edge_list = [i + tuple([0.1]) for i in edge_list]  # ウェイトの値を適当に追加

# jupyter中に表示させる場合、notebook=True, →にする場合directed=True
got_net = net.Network(notebook=True, directed=True)

for e in edge_list:
    src = e[0]
    dst = e[1]
    w = e[2]

    got_net.add_node(src, title=src, shape='ellipse', label=src)
    got_net.add_node(dst, title=dst, shape='ellipse', label=dst)
    got_net.add_edge(src, dst, width=3)

    # print(src)
    # print(dst)
    # print(w)
    # print('='*10)

neighbor_map = got_net.get_adj_list()

for node in got_net.nodes:
    node['title'] += ' Neighbors:<br>' + '<br>'.join(neighbor_map[node['id']])
    node['value'] = len(neighbor_map[node['id']])
    # node['label'] = node['label']

got_net.show_buttons(False)

got_net.show('../work/output/network.html')
